In [36]:
import os
import sys
import uuid

pipeline_dir = os.path.join(os.getcwd(), '..')
if pipeline_dir not in sys.path:
    sys.path.append(pipeline_dir)

In [37]:
from pipeline import Pipeline, events, resources

# Resources
Resources represent the AWS resources (SQS, SNS, DynamoDB etc.) used by your pipeline.
## Defining Resources
You can define a resource by creating a new class inheriting from the appropriate resources object

In [57]:
def _getattr(obj, name):
    try:
        return getattr(obj, name)()
    except:
        pass

print([x for x in dir(resources) if hasattr(_getattr(resources, x), 'name')])

['DynamoDB', 'S3Bucket', 'SNSPolicy', 'SNSTopic', 'SQSPolicy', 'SQSQueue', 'ServerlessResource']


In [58]:
# Creating a SNS Topic
class MyTopic(resources.SNSTopic):
    
    def __init__(self):
        super().__init__()

my_topic = MyTopic()
print(my_topic)

{'Type': 'AWS::SNS::Topic', 'Properties': {'TopicName': 'MyTopic'}}


## Customizing Resources

The resource is a wrapper of its respective CloudFormation template.  The template, by default, contains the minimum required information to create a valid AWS resource.  You can customize your resource's CloudFormation template using it's dict interface:

In [59]:
# Creating a SNS Topic
class MyTopic(resources.SNSTopic):
    
    def __init__(self):
        super().__init__()
        # Adding a display name
        self['Properties'].update({'DisplayName': 'MyDisplayName'})

my_topic = MyTopic()
print(my_topic)

{'Type': 'AWS::SNS::Topic', 'Properties': {'TopicName': 'MyTopic', 'DisplayName': 'MyDisplayName'}}


Some resources inherit helper methods used to create the resource.

In [60]:
class MyTable(resources.DynamoDB):

    def __init__(self):
        super().__init__()
        # Define an attribute and primary key field
        self.add_attribute('id', 'S')
        self.add_key('id', 'HASH')

my_table = MyTable()
print(my_table)

{'Type': 'AWS::DynamoDB::Table', 'Properties': {'TableName': 'MyTable', 'AttributeDefinitions': [{'AttributeName': 'id', 'AttributeType': 'S'}], 'KeySchema': [{'AttributeName': 'id', 'KeyType': 'HASH'}], 'ProvisionedThroughput': {'ReadCapacityUnits': 1, 'WriteCapacityUnits': 1}}}


## Utilizing Resources
To use a resource in your pipeline, instantiate the class and pass into your pipeline

In [61]:
class MyPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(resources=[my_topic, my_table])

Resources can be the target of event triggers

In [62]:
class MyPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(resources=[my_topic, my_table])
    
    @events.sns(resource=my_topic)
    def sns_trigger(self, event, context):
        print(event)

Some resources inherit helper methods for interacting with the resource.

In [63]:
class MyPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(resources=[my_topic, my_table])
    
    @events.sns(resource=my_topic)
    def sns_trigger(self, event, context):
        # Log event in DynamoDB table
        my_table.put(event)